# Problem Introduction

Our task is to build a model, which is suitable for recommending books based on user reviews from open BX dataset.
So lets tackle this problem using `numpy`, `pandas` and `sklearn`

In [23]:
import numpy as np
import pandas as pd

# Exploratory Data Analysis

In [24]:
# input is encoded in latin-1 and there are come corrupted values
# as there is not many of those values, I decided to just skip them
books_df = pd.read_csv( "BX-Books.csv", encoding="latin-1", sep=";", on_bad_lines="skip", low_memory=False )
book_ratings_df = pd.read_csv( "BX-Book-Ratings.csv", encoding="latin-1", sep=";", on_bad_lines="skip" )
book_users_df = pd.read_csv( "BX-Users.csv", encoding="latin-1", sep=";", on_bad_lines="skip" )